In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv('births_toy.csv')

In [10]:
df.shape

(1000, 56)

In [11]:
df.head()

,Unnamed: 0,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,1,26.6,0,2570,12,2017,2.0,9,...,0,0,1,110,1,2,N,0,M,64
1,2,1,1,24.1,0,3410,8,2017,1.0,5,...,0,0,1,224,1,2,N,0,M,45
2,3,1,1,28.8,0,2041,6,2018,2.0,7,...,0,0,0,153,1,1,N,0,F,36
3,4,1,1,31.6,0,4054,8,2018,2.0,1,...,0,1,0,155,1,2,N,0,M,0
4,5,1,1,34.4,0,3850,4,2017,1.0,5,...,0,2,0,106,1,1,N,0,F,25
